In [1]:
%install-location $cwd/swift-install
%install '.package(path: "$cwd/SchwiftyNotebook_matrix_multiplication")' SchwiftyNotebook_matrix_multiplication

Installing packages:
	.package(path: "/home/ubuntu/workspace/fast-ai-swift/SchwiftyNotebook_matrix_multiplication")
		SchwiftyNotebook_matrix_multiplication
With SwiftPM flags: []
Working in: /tmp/tmpnmh58mvl/swift-install
/home/ubuntu/swift/usr/bin/swift: /home/ubuntu/anaconda3/envs/swift-env/lib/libuuid.so.1: no version information available (required by /home/ubuntu/swift/usr/bin/swift)
/home/ubuntu/swift/usr/bin/swift: /home/ubuntu/anaconda3/envs/swift-env/lib/libuuid.so.1: no version information available (required by /home/ubuntu/swift/usr/bin/swift)
/home/ubuntu/swift/usr/bin/swift: /home/ubuntu/anaconda3/envs/swift-env/lib/libuuid.so.1: no version information available (required by /home/ubuntu/swift/usr/bin/swift)
/home/ubuntu/swift/usr/bin/swift: /home/ubuntu/anaconda3/envs/swift-env/lib/libuuid.so.1: no version information available (required by /home/ubuntu/swift/usr/bin/swift)
/home/ubuntu/swift/usr/bin/swift: /home/ubuntu/anaconda3/envs/swift-env/lib/libuuid.so.1: no vers

In [2]:
//export
import Path
import TensorFlow
import SchwiftyNotebook_matrix_multiplication

In [3]:
let thingo = Tensor([-2, -1, 0, 1, 2])

In [82]:
time(repeating: 2){ relu(thingo) }

average: 0.1859025 ms,   min: 0.168788 ms,   max: 0.203017 ms


In [67]:
//export
public func reLU<T>(tensor: Tensor<T>) -> Tensor<T> where T : FloatingPoint, T: TensorFlowScalar {
    return max(tensor, 0)
}

In [6]:
//export
public func linearCombination<T>(inputs: Tensor<T>, weights: Tensor<T>, bias: Tensor<T>) -> Tensor<T> where T: FloatingPoint, T: TensorFlowScalar {
    return matmul(inputs, weights) + bias
}

# Normalization

In [7]:
let (xTrainingData, yTrainingData, xValidationData, yValidationData) = loadMNISTData(path: mnistPath)

In [8]:
let xTrainingDataNormalized = normalizeTensor(tensor: xTrainingData)
                                        .reshaped(to: [xTrainingData.shape[0], 784])
let xValidationDataNormalized = normalizeTensor(tensor: xValidationData)
                                        .reshaped(to: [xValidationData.shape[0], 784])

In [9]:
// export
public typealias TensorFloat=Tensor<Float>
public func assertNearZero(_ tensor: TensorFloat, _ threshold: Float = 1e-3){
    assert(tensor < threshold, "Expected \(tensor) to be less than \(threshold)")
}

In [10]:
assertNearZero(xTrainingDataNormalized.mean())
assertNearZero(xValidationDataNormalized.mean())
assertNearZero(1 - xTrainingDataNormalized.standardDeviation())
assertNearZero(1 - xValidationDataNormalized.standardDeviation())

In [11]:
let (numberOfImages, numberOfPixels) = (xTrainingDataNormalized.shape[0], xTrainingDataNormalized.shape[1])
let numberOfClasses = 10
let layerOutput = 50
print(numberOfImages, numberOfPixels, numberOfClasses)

60000 784 10


# Initialization

## From Hand

In [12]:
let parameterLayerOne = TensorFloat(randomNormal: [numberOfPixels, layerOutput]) / sqrt(Float(numberOfPixels))
let parameterLayerTwo = TensorFloat(randomNormal: [layerOutput, 1]) / sqrt(Float(layerOutput))

In [14]:
assertNearZero(parameterLayerOne.mean())
assertNearZero(parameterLayerOne.standardDeviation() - 1 / sqrt(Float(numberOfPixels)))

In [16]:
let biasLayerOne = TensorFloat(zeros: [layerOutput])
let biasLayerTwo = TensorFloat(zeros: [1])

In [17]:
assertNearZero(biasLayerOne.mean())
assertNearZero(biasLayerOne.standardDeviation() - 1 / sqrt(Float(numberOfImages)))

In [18]:
print(xValidationDataNormalized.shape, parameterLayerOne.shape, biasLayerOne.shape)

[10000, 784] [784, 50] [50]


### Side Adventure: Timing

In [19]:
//export
import Dispatch

func getTimeUnit(_ nanoSeconds: Double) -> String {
    let powerOfTen = floor(log10(nanoSeconds))
    switch powerOfTen {
        case 1..<3:
            return "\(nanoSeconds) ns"
        case 3..<6:
            return "\(nanoSeconds/1e3) µs"
        case 6..<9: 
            return "\(nanoSeconds/1e6) ms"
        default: 
            return "\(nanoSeconds/1e9) s"
    }
}

// ⏰Time how long it takes to run the specified function, optionally taking
// the average across a number of repetitions.
public func withTime<T>(_ f: () -> T) -> T {
    let start = DispatchTime.now()
    let value = f()
    let end = DispatchTime.now()
    let nanoSeconds = Double(end.uptimeNanoseconds - start.uptimeNanoseconds)
    print("elapsed time: \(getTimeUnit(nanoSeconds))")
    return value
}

In [20]:
withTime{
    print("yeet")
}

yeet
elapsed time: 29.743 µs


In [21]:
let linearWomboCombo = time(repeating: 10) {
    withDevice(.gpu){
        linearCombination(inputs: xValidationDataNormalized, 
                                         weights: parameterLayerOne, 
                                         bias: biasLayerOne)
    }
}

average: 0.055230800000000004 ms,   min: 0.051097 ms,   max: 0.073791 ms


In [22]:
let cpuLinearWomboCombo = time(repeating: 10) {
    withDevice(.cpu){
        linearCombination(inputs: xValidationDataNormalized, 
                                         weights: parameterLayerOne, 
                                         bias: biasLayerOne)
    }
}

average: 16.3492737 ms,   min: 12.51066 ms,   max: 30.64353 ms


In [23]:
let defaultLinearWomboCombo = time(repeating: 10) {
    linearCombination(inputs: xValidationDataNormalized, 
                                     weights: parameterLayerOne, 
                                     bias: biasLayerOne)
}

average: 0.0437552 ms,   min: 0.041054 ms,   max: 0.051263 ms


In [24]:
let thing = withTime { linearCombination(inputs: xValidationDataNormalized, 
                                         weights: parameterLayerOne, 
                                         bias: biasLayerOne) }

elapsed time: 178.859 µs


---

In [25]:
let linearWomboCombo = withTime { linearCombination(inputs: xValidationDataNormalized, 
                                         weights: parameterLayerOne, 
                                         bias: biasLayerOne) }

elapsed time: 178.888 µs


In [26]:
(linearWomboCombo.mean(), linearWomboCombo.standardDeviation())

▿ 2 elements
  - .0 : 0.057206873
  - .1 : 1.0378091


In [30]:
import Python
%include "EnableIPythonDisplay.swift"
IPythonDisplay.shell.enable_matplotlib("inline")

('inline', 'module://ipykernel.pylab.backend_inline')


In [31]:
public let plotter = Python.import("matplotlib.pyplot")

In [43]:
linearWomboCombo.shape

▿ [10000, 50]
  ▿ dimensions : 2 elements
    - 0 : 10000
    - 1 : 50


In [32]:
let activations = withTime { reLU(tensor: linearWomboCombo) }

elapsed time: 353.830804 ms


In [28]:
print(activations.mean(), activations.standardDeviation()) //relu zeros out all of the stuff below zero (so only ~half remain)

0.43751225 0.6271196


### Kaiming Initialization

In [47]:
let kaimingInitParamOne = TensorFloat(randomNormal: [numberOfPixels, layerOutput]) * (2.0/sqrt(Float(numberOfPixels)))

In [48]:
(kaimingInitParamOne.mean(), kaimingInitParamOne.standardDeviation())

▿ 2 elements
  - .0 : 0.00034293102
  - .1 : 0.07114902


In [49]:
let kaimingWomboCombo = withTime{ linearCombination(inputs: xValidationDataNormalized, 
                                          weights: kaimingInitParamOne, 
                                          bias: biasLayerOne ) }

elapsed time: 196.456 µs


In [50]:
(kaimingWomboCombo.mean(), kaimingWomboCombo.standardDeviation())

▿ 2 elements
  - .0 : 0.071222365
  - .1 : 1.9520401


In [51]:
let kaimingActivations = reLU(tensor: kaimingWomboCombo)

In [52]:
(kaimingActivations.mean(), kaimingActivations.standardDeviation())

▿ 2 elements
  - .0 : 0.80600166
  - .1 : 1.1343071


### Basic Model

In [53]:
let kaimingParamOne = TensorFloat(randomNormal: [numberOfPixels, layerOutput]) * (2.0/sqrt(Float(numberOfPixels)))
let kaimingParamTwo = TensorFloat(randomNormal: [layerOutput, 1]) * (2.0/sqrt(Float(layerOutput)))
let biasLayerOne = TensorFloat(zeros: [layerOutput])
let biasLayerTwo = TensorFloat(zeros: [1])

In [69]:
func basicBitch(_ inputTensor: TensorFloat) -> TensorFloat {
    let firstParams = withTime { linearCombination(inputs: inputTensor, 
                                                          weights: kaimingParamOne, 
                                                          bias: biasLayerOne) }
    let firstActivations = withTime { reLU(tensor: firstParams)}
    return withTime{ linearCombination(inputs: firstActivations, 
                             weights: kaimingParamTwo, 
                             bias: biasLayerTwo) }
}

In [83]:
let prediction = basicBitch(xValidationDataNormalized)

elapsed time: 184.576 µs
elapsed time: 353.584292 ms
elapsed time: 114.75 µs
elapsed time: 75.322 µs
elapsed time: 1.016848 ms
elapsed time: 70.84 µs


🤔 Why is reLU so sloooow???

Turns out its just slow the first time...

In [84]:
for i in 1...10 {
 withTime { relu(kaimingWomboCombo)}   
}

elapsed time: 353.924038 ms
elapsed time: 195.184 µs
elapsed time: 136.994 µs
elapsed time: 123.791 µs
elapsed time: 97.164 µs
elapsed time: 93.909 µs
elapsed time: 92.597 µs
elapsed time: 93.244 µs
elapsed time: 91.573 µs
elapsed time: 95.022 µs
